<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/Fasttext1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unidecode
!pip install word2number
!pip install contractions

  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5582 sha256=beff3aea39d80836f467cac9c7d0b1a0d2525996dd51696290e490707b2583f5
  Stored in directory: /root/.cache/pip/wheels/4b/c3/77/a5f48aeb0d3efb7cd5ad61cbd3da30bbf9ffc9662b07c9f879
Successfully built word2number
     |████████████████████████████████| 284 kB 8.1 MB/s 
     |████████████████████████████████| 321 kB 66.8 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85452 sha256=ac55d29d01d703d087d88315a9bf7a8bafba6aa7f47f095315e1fe5af53cd6e6
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
import gensim
from gensim.models import Word2Vec
import string
string.punctuation
nltk.download('stopwords')
import re
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import contractions
from nltk.tokenize.treebank import TreebankWordDetokenizer
# load spacy model, can be "en_core_web_sm" as well
from sklearn.model_selection import train_test_split
import xgboost as xgb



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [262]:
df_fake = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_fake.csv')
df_real = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_real.csv')
df_fake['labelML']=0
df_real['labelML']=1

df_fake.count()


id           5323
news_url     5067
title        5323
tweet_ids    5135
labelML      5323
dtype: int64

In [263]:
#df= df_fake.head(50).append(df_real.head(50))
df= df_fake.append(df_real)
df.reset_index(drop=True, inplace=True)

In [264]:
print(df.columns)
df.count()
print(df)

Index(['id', 'news_url', 'title', 'tweet_ids', 'labelML'], dtype='object')
                         id  ... labelML
0      gossipcop-2493749932  ...       0
1      gossipcop-4580247171  ...       0
2       gossipcop-941805037  ...       0
3      gossipcop-2547891536  ...       0
4      gossipcop-5476631226  ...       0
...                     ...  ...     ...
22135      gossipcop-891749  ...       1
22136      gossipcop-941486  ...       1
22137      gossipcop-953143  ...       1
22138      gossipcop-954366  ...       1
22139      gossipcop-877062  ...       1

[22140 rows x 5 columns]


In [265]:
print(df)

                         id  ... labelML
0      gossipcop-2493749932  ...       0
1      gossipcop-4580247171  ...       0
2       gossipcop-941805037  ...       0
3      gossipcop-2547891536  ...       0
4      gossipcop-5476631226  ...       0
...                     ...  ...     ...
22135      gossipcop-891749  ...       1
22136      gossipcop-941486  ...       1
22137      gossipcop-953143  ...       1
22138      gossipcop-954366  ...       1
22139      gossipcop-877062  ...       1

[22140 rows x 5 columns]


In [266]:
df1= df.drop(['tweet_ids', 'news_url','id'], axis=1)
df1.count()

title      22140
labelML    22140
dtype: int64

In [267]:
# NLP Preprocessing
from gensim.utils import simple_preprocess

# NLP Preprocess - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15)[source]
# Convert a document into a list of tokens.
# This lowercases, tokenizes, de-accents (optional). – the output are final tokens = unicode strings, that won’t be processed any further.

df1.iloc[:, 0] = df1.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))

In [210]:
# Prefixing each row of the category column with '__label__' 0;10 all rows. same for column. 
#---------df1.iloc[:, 0] = df1.iloc[1:, 0].apply(lambda x: '__label__' + x)
#print(df1)

In [268]:
#print(df1)
df1.count()

title      22140
labelML    22140
dtype: int64


 #df = df.loc[:,"title"]
 # just for loc practice, ignore.

 X_train - This includes your all independent variables,these will be used to train the model, also as we have specified the test_size = 0.4, this means 60% of observations from your complete data will be used to train/fit the model and rest 40% will be used to test the model.

2). X_test - This is remaining 40% portion of the independent variables from the data which will not be used in the training phase and will be used to make predictions to test the accuracy of the model.

3). y_train - This is your dependent variable which needs to be predicted by this model, this includes category labels against your independent variables, we need to specify our dependent variable while training/fitting the model.

4). y_test - This data has category labels for your test data, these labels will be used to test the accuracy between actual and predicted categories.


In [269]:
# Prefixing each row of the category column with '__label__' 0;10 all rows. same for column. 
df1.iloc[:5322, 0] = df1.iloc[:, 0].apply(lambda x: '__label__0 ' + x)
df1.iloc[5323:, 0] = df1.iloc[1:, 0].apply(lambda x: '__label__1 ' + x)


In [270]:
print(df1)

                                                   title  labelML
0      __label__0 did miley cyrus and liam hemsworth ...        0
1      __label__0 paris jackson cara delevingne enjoy...        0
2      __label__0 celebrities join tax march in prote...        0
3      __label__0 cindy crawford daughter kaia gerber...        0
4      __label__0 full list of oscar nominations variety        0
...                                                  ...      ...
22135  __label__1 hollywood film awards the complete ...        1
22136  __label__1 jada pinkett smith explains why son...        1
22137  __label__1 tinsley mortimer reacts to luann de...        1
22138  __label__1 prince harry carries on princess di...        1
22139  __label__1 kylie jenner is actually terrified ...        1

[22140 rows x 2 columns]


In [271]:
df1.count()

title      22140
labelML    22140
dtype: int64

In [272]:
df1.reset_index(drop=True, inplace=True)
X, y = df1.iloc[:, 1:], df1['labelML']

# Create the training and test sets #IMP: use stratify when dataset is not balanced 
#https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas 
#--for few entries-- #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3)

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train, y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds == y_test)) / y_test.shape[0]
print("accuracy: %f" % (accuracy))

accuracy: 1.000000


In [273]:
df2=df1
# ------------- df2 = df2.reindex(columns=['labelML', 'title'])
#print(df1)

In [274]:
traindf, testdf = train_test_split(df2, test_size=0.2)
print(traindf)
print(testdf)  #check things here since it is kinda wrong.

traindf.count()

                                                   title  labelML
11016  __label__1 simon cowell rushed to hospital aft...        1
5807   __label__1 busy philipps responds to backlash ...        1
17607  __label__1 real housewives of potomac star mon...        1
4869   __label__0 star girly princess suri cruise vs ...        0
11580  __label__1 johnny depp star in crisis and the ...        1
...                                                  ...      ...
15189                       __label__1 we day california        1
20471  __label__1 kevin spacey replaced by christophe...        1
2360   __label__0 is brad pitt body odor becoming pro...        0
14326  __label__1 have adam sandler drew barrymore ev...        1
9655   __label__1 kelly clarkson responds to grammys ...        1

[17712 rows x 2 columns]
                                                   title  labelML
5045   __label__0 katy perry and orlando bloom cozy u...        0
17950  __label__1 cancer destroyed tristan butt bu

title      17712
labelML    17712
dtype: int64

In [275]:
# Saving the CSV file as a text file to train/test the classifier
import csv #needed
traindf[['labelML', 'title']].to_csv('traindf.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

testdf[['labelML', 'title']].to_csv('testdf.txt', 
                                     index = False, 
                                     sep = ' ',
                                     header = None, 
                                     quoting = csv.QUOTE_NONE, 
                                     quotechar = "", 
                                     escapechar = " ")

In [276]:
!pip install fasttext
import fasttext
# Training the fastText classifier
model = fasttext.train_supervised('traindf.txt', wordNgrams = 2)

# Evaluating performance on the entire test file
model.test('testdf.txt')                      

# Predicting on a single input
testdf['labelML'] = testdf['labelML'].astype(str)
model.predict(testdf.iloc[56, 0])
#model.test(testdf.txt)

# Save the trained model
#model.save_model('model.bin')

(('__label__1',), array([0.84595311]))

In [277]:
testdf.head(50)
#print(testdf.columns)

,title,labelML
5045,__label__0 katy perry and orlando bloom cozy u...,0
17950,__label__1 cancer destroyed tristan butt but w...,1
3130,__label__0 britney spears speaks pop star talk...,0
9167,__label__1 star wars fans show up for solo red...,1
2478,__label__0 blake shelton reveals that he misse...,0
16454,__label__1 vanderpump rules ariana madix recal...,1
11653,__label__1 robert henstridge,1
15666,__label__1 see every celebrity at the royal we...,1
19586,__label__1 meghan markle dad thomas causing ro...,1
1802,__label__0 selena gomez and justin bieber hang...,0


This model is running, so trying to change with vectors, or textual input for new vectors. Accuracy is only 50 percent coz I've given it such a balanced dataset.

In [ ]:
# Predicting on a single input
#model.predict(test.iloc['Miley cyrus is new'])
#use this later when we have the representations I guess. 
